In [ ]:
cur = con.cursor()

cur.execute('SELECT (SELECT AVG(popularity) as popularity FROM songs WHERE instrumentalness >= 0.9) AS instrumental_popularity,  (SELECT AVG(popularity) as popularity FROM songs) AS avg_popularity;')
rows = cur.fetchall()

1- Establish a connection between Python and the Sakila database.

In [2]:
import sqlite3
con = sqlite3.connect('db/sakila.db')
con

2- Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- month: an integer representing the month for which rental data is to be retrieved.
- year: an integer representing the year for which rental data is to be retrieved.
 
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [26]:
import pandas as pd
def rentals_month(engine, month, year):
    cur = engine.cursor()
    cur.execute("SELECT * FROM rental WHERE CAST(strftime('%m', date(rental_date)) AS INT) = ?  AND CAST(strftime('%Y', date(rental_date)) AS INT) = ?", [month, year])
    rows = cur.fetchall()

    return pd.DataFrame(rows, columns=[description[0] for description in cur.description])

In [32]:
# Test the function
rentals_month(con, 6, 2005)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33.000,1632,416,2005-06-18 21:37:33.000,2,2021-03-06 15:53:50
1,1159,2005-06-14 22:55:13.000,4395,516,2005-06-17 02:11:13.000,1,2021-03-06 15:53:50
2,1160,2005-06-14 23:00:34.000,2795,239,2005-06-18 01:58:34.000,2,2021-03-06 15:53:50
3,1161,2005-06-14 23:07:08.000,1690,285,2005-06-21 17:12:08.000,1,2021-03-06 15:53:50
4,1162,2005-06-14 23:09:38.000,987,310,2005-06-23 22:00:38.000,1,2021-03-06 15:53:50
...,...,...,...,...,...,...,...
2306,3465,2005-06-21 22:10:01.000,1488,510,2005-06-30 21:35:01.000,1,2021-03-06 15:54:09
2307,3466,2005-06-21 22:13:33.000,371,226,2005-06-25 21:01:33.000,2,2021-03-06 15:54:09
2308,3467,2005-06-21 22:19:25.000,729,543,2005-06-27 00:03:25.000,2,2021-03-06 15:54:09
2309,3468,2005-06-21 22:43:45.000,2899,100,2005-06-30 01:49:45.000,1,2021-03-06 15:54:09


3- Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()


In [55]:
def rental_count_month(rental_months, month, year):
    df = rental_months[["customer_id", "rental_id"]].groupby("customer_id").agg("count")
    df.columns = ["rentals_"+(str(month) if len(str(month)) == 2 else "0"+str(month) )+"_"+str(year)]
    return df

In [56]:
#Test
year = 2005
month = 5
rental_count_month(rentals_month(con, month, year), month, year)

,rentals_05_2005
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


4- Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [75]:
def compare_rentals(df1, df2):
    df3 = df1.merge(df2, how="inner", left_index=True, right_index=True)
    df3["difference"] = df3[df3.columns[1]] - df3[df3.columns[0]]

    return df3

In [ ]:
# Test
compare_rentals(rental_count_month(rentals_month(con, 5, 2005),  5, 2005), 
                rental_count_month(rentals_month(con,  6, 2005),  6, 2005))

,rentals_05_2005,rentals_06_2005,difference
customer_id,,,
1,2,7,5
2,1,1,0
3,2,4,2
5,3,5,2
6,3,4,1
...,...,...,...
594,4,6,2
595,1,2,1
596,6,2,-4
